In [16]:
# Import required libraries
import sqlite3
import pandas as pd

In [17]:
%load_ext sql

%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

In [18]:
# Connecting to the database
%sql sqlite:///vivino.db

In [19]:
%%sql
-- Lists all the tables in a particular database
SELECT name FROM sqlite_master WHERE type='table';

Running query in 'sqlite:///vivino.db'

name
countries
grapes
wineries
flavor_groups
keywords
regions
most_used_grapes_per_country
toplists
wines
vintages


In [20]:
# %%sql
# DROP TABLE IF EXISTS flavor_groups

In [21]:
# %%sql
# DROP TABLE IF EXISTS wineries

## We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

In [60]:
%%sql
SELECT vintages.name AS wine_name,
       MIN(vintages.year) AS year,
       MIN(vintages.price_euros) AS price_euros,
       MAX(vintages.ratings_average) AS ratings_average,
       MAX(wines.ratings_count) AS ratings_count
FROM vintages
JOIN wines ON vintages.wine_id = wines.id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE vintages.price_euros < 300
GROUP BY wines.name
ORDER BY MIN(vintage_toplists_rankings.rank) ASC
LIMIT 10;

Running query in 'sqlite:///vivino.db'

wine_name,year,price_euros,ratings_average,ratings_count
San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018,2017,24.75,4.5,94289
Tommasi Amarone della Valpolicella Classico 2018,2017,49.8,4.4,77515
Buena Vista Chateau Buena Vista Cabernet Sauvignon Napa Valley 2018,2018,61.95,4.4,13415
Guerrieri Guerriero della Terra 2019,2019,27.4,4.4,10185
Charles Melton Nine Popes 2015,2015,114.6,4.4,2118
Bodegas Tradición Palo Cortado Tradición Vors 30 Years Sherry,N.V.,99.95,4.4,1050
Arzuaga Reserva Ribera del Duero 2018,2018,39.55,4.4,11233
Pago de Carraovejas Tinto 2020,2019,42.95,4.5,65625
Château des Tours Vacqueyras 2011,2011,250.0,4.4,3412
Dal Forno Romano Valpolicella Superiore Monte Lodoletta 2004,2004,243.75,4.5,13024


In [61]:
# Connect to SQLite database
conn = sqlite3.connect(r'./vivino.db')

query1 = """
SELECT vintages.name AS wine_name,
    MIN(vintages.year) AS year,
    MIN(vintages.price_euros) AS price_euros,
    MAX(vintages.ratings_average) AS ratings_average,
    MAX(wines.ratings_count) AS ratings_count
FROM
    vintages
JOIN wines
    ON vintages.wine_id = wines.id
JOIN vintage_toplists_rankings 
    ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE 
    vintages.price_euros < 300
GROUP BY
    wines.name
ORDER BY MIN(vintage_toplists_rankings.rank) ASC
LIMIT 10;
"""


In [63]:
# Execute the query and load the results into a pandas DataFrame
sales_df = pd.read_sql_query(query1, conn)

# Write the DataFrame to a CSV file
sales_df.to_csv('wine_top_10.csv', index=False)

# Optional: Display the first few rows of the DataFrame
print(sales_df.head())

                                           wine_name  year  price_euros  \
0  San Marzano 60 Sessantanni Old Vines Primitivo...  2017        24.75   
1   Tommasi Amarone della Valpolicella Classico 2018  2017        49.80   
2  Buena Vista Chateau Buena Vista Cabernet Sauvi...  2018        61.95   
3               Guerrieri Guerriero della Terra 2019  2019        27.40   
4                     Charles Melton Nine Popes 2015  2015       114.60   

   ratings_average  ratings_count  
0              4.5          94289  
1              4.4          77515  
2              4.4          13415  
3              4.4          10185  
4              4.4           2118  


## We have a limited marketing budget for this year. Which country should we prioritise and why?

## We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?

## We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

## We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.

## We would like to create a country leaderboard. Come up with a visual that shows the average wine rating for each country. Do the same for the vintages.

## One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?

## Test code

In [35]:
%%sql
SELECT wines.name, wines.ratings_average as wine_rating_Avg, 
wines.ratings_count as wine_ratings_count, 
vintages.price_euros as vintage_price, 
vintages.ratings_count as vintage_rating_count
FROM wines
JOIN vintages ON wines.id = vintages.wine_id
WHERE wines.name = 'Brut Champagne'
GROUP BY vintage_rating_count
ORDER BY vintage_rating_count DESC

Running query in 'sqlite:///vivino.db'

name,wine_rating_Avg,wine_ratings_count,vintage_price,vintage_rating_count
Brut Champagne,4.6,146377,440.0,21121
Brut Champagne,4.6,146377,460.0,14650
Brut Champagne,4.6,146377,397.5,6496
Brut Champagne,4.6,146377,638.83,5945
Brut Champagne,4.6,146377,250.0,623


In [50]:
%%sql
SELECT wines.name AS wine_name,
       MIN(vintages.year) AS year,
       MIN(vintages.price_euros) AS price_euros,
       MAX(vintages.ratings_average) AS ratings_average,
       MAX(wines.ratings_count) AS ratings_count
FROM vintages
JOIN wines ON vintages.wine_id = wines.id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
WHERE vintages.price_euros < 300
GROUP BY wines.name
ORDER BY MIN(vintage_toplists_rankings.rank) ASC
LIMIT 10;

Running query in 'sqlite:///vivino.db'

wine_name,year,price_euros,ratings_average,ratings_count
60 Sessantanni Old Vines Primitivo di Manduria,2017,24.75,4.5,94289
Amarone della Valpolicella Classico,2017,49.8,4.4,77515
Chateau Buena Vista Cabernet Sauvignon Napa Valley,2018,61.95,4.4,13415
Guerriero della Terra,2019,27.4,4.4,10185
Nine Popes,2015,114.6,4.4,2118
Palo Cortado Tradición Vors 30 Years Sherry,N.V.,99.95,4.4,1050
Reserva Ribera del Duero,2018,39.55,4.4,11233
Tinto,2019,42.95,4.5,65625
Vacqueyras,2011,250.0,4.4,3412
Valpolicella Superiore Monte Lodoletta,2004,243.75,4.5,13024
